In [5]:
fraction_of_sessions_to_use = 1

import pandas as pd
import numpy as np

train = pd.read_parquet('../data/train.parquet').head(1000)
test = pd.read_parquet('../data/test.parquet').head(1000)

sample_sub = pd.read_csv('../data/sample_submission.csv')

if fraction_of_sessions_to_use != 1:
    lucky_sessions_train = train.drop_duplicates(['session']).sample(frac=fraction_of_sessions_to_use, random_state=42)['session']
    subset_of_train = train[train.session.isin(lucky_sessions_train)]
    
    lucky_sessions_test = test.drop_duplicates(['session']).sample(frac=fraction_of_sessions_to_use, random_state=42)['session']
    subset_of_test = test[test.session.isin(lucky_sessions_test)]
else:
    subset_of_train = train
    subset_of_test = test

subset_of_train.index = pd.MultiIndex.from_frame(subset_of_train[['session']])
subset_of_test.index = pd.MultiIndex.from_frame(subset_of_test[['session']])

chunk_size = 30_000
min_ts = train.ts.min()
max_ts = test.ts.max()

from collections import defaultdict, Counter
next_AIDs = defaultdict(Counter)

subsets = pd.concat([subset_of_train, subset_of_test])
sessions = subsets.session.unique()

In [23]:
subsets

,session,aid,ts,type
session,,,,
0,0,1517085,1659304800,0
0,0,1563459,1659304904,0
0,0,1309446,1659367439,0
0,0,16246,1659367719,0
0,0,1781822,1659367871,0
...,...,...,...,...
12899852,12899852,701401,1661730832,0
12899852,12899852,701401,1661730866,0
12899852,12899852,1579739,1661730906,0


In [25]:
len(sessions)

88

In [26]:
next_AIDs

defaultdict(collections.Counter, {})

In [29]:
subsets.loc[sessions[i]:sessions[min(sessions.shape[0]-1, i+chunk_size-1)]].reset_index(drop=True)

,session,aid,ts,type
0,0,1517085,1659304800,0
1,0,1563459,1659304904,0
2,0,1309446,1659367439,0
3,0,16246,1659367719,0
4,0,1781822,1659367871,0
...,...,...,...,...
1995,12899852,701401,1661730832,0
1996,12899852,701401,1661730866,0
1997,12899852,1579739,1661730906,0
1998,12899852,372283,1661730988,0


In [39]:
subsets.loc[sessions[i]:sessions[min(sessions.shape[0]-1, i+chunk_size-1)]].reset_index(drop=True).groupby('session', as_index=False).nth(list(range(-30,0))).reset_index(drop=True).head(50)

,session,aid,ts,type
0,0,543308,1661552940,2
1,0,961113,1661562901,0
2,0,883849,1661563120,0
3,0,701766,1661586370,0
4,0,924751,1661586382,0
5,0,168206,1661586401,0
6,0,924751,1661591004,0
7,0,219033,1661593840,0
8,0,171982,1661593852,0
9,0,1319939,1661607646,0


In [40]:
subsets.loc[sessions[i]:sessions[min(sessions.shape[0]-1, i+chunk_size-1)]].reset_index(drop=True).groupby('session', as_index=False).nth(list(range(-30,0))).reset_index(drop=True).merge(current_chunk, on='session').head(40)

,session,aid_x,ts_x,type_x,aid_y,ts_y,type_y
0,0,543308,1661552940,2,543308,1661552940,2
1,0,543308,1661552940,2,961113,1661562901,0
2,0,543308,1661552940,2,883849,1661563120,0
3,0,543308,1661552940,2,701766,1661586370,0
4,0,543308,1661552940,2,924751,1661586382,0
5,0,543308,1661552940,2,168206,1661586401,0
6,0,543308,1661552940,2,924751,1661591004,0
7,0,543308,1661552940,2,219033,1661593840,0
8,0,543308,1661552940,2,171982,1661593852,0
9,0,543308,1661552940,2,1319939,1661607646,0


In [42]:
subsets.loc[sessions[i]:sessions[min(sessions.shape[0]-1, i+chunk_size-1)]].reset_index(drop=True).groupby('session', as_index=False).nth(list(range(-30,0))).reset_index(drop=True).merge(current_chunk, on='session')

/var/folders/31/jtc_3nm57535kt99w8nb24lh0000gn/T/ipykernel_88451/600098579.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  subsets.loc[sessions[i]:sessions[min(sessions.shape[0]-1, i+chunk_size-1)]].reset_index(drop=True).groupby('session', as_index=False).nth(list(range(-30,0))).reset_index(drop=True).merge(current_chunk, on='session')[consecutive_AIDs.aid_x != consecutive_AIDs.aid_y]


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [44]:
 consecutive_AIDs

,session,aid_x,ts_x,type_x,aid_y,ts_y,type_y,days_elapsed
1,0,543308,1661552940,2,961113,1661562901,0,0.115289
2,0,543308,1661552940,2,883849,1661563120,0,0.117824
3,0,543308,1661552940,2,701766,1661586370,0,0.386921
4,0,543308,1661552940,2,924751,1661586382,0,0.387060
5,0,543308,1661552940,2,168206,1661586401,0,0.387280
...,...,...,...,...,...,...,...,...
20540,12899852,701401,1661730866,0,372283,1661730988,0,0.001412
20541,12899852,701401,1661730866,0,199292,1661731011,0,0.001678
20564,12899852,1579739,1661730906,0,372283,1661730988,0,0.000949
20565,12899852,1579739,1661730906,0,199292,1661731011,0,0.001215


In [45]:
next_AIDs

defaultdict(collections.Counter,
            {543308: Counter({961113: 1,
                      883849: 1,
                      701766: 1,
                      924751: 2,
                      168206: 1,
                      219033: 1,
                      171982: 1,
                      1319939: 1,
                      1349536: 3,
                      334392: 2,
                      165096: 1,
                      315914: 2,
                      1680276: 1,
                      1818905: 1,
                      1048797: 1,
                      341626: 1,
                      219925: 1,
                      843110: 1,
                      938007: 1,
                      1228848: 1,
                      1740927: 1,
                      161938: 1}),
             961113: Counter({883849: 1,
                      701766: 1,
                      924751: 2,
                      168206: 1,
                      219033: 1,
                      171982: 1,
                  

In [47]:
test_session_AIDs

session
12899779                                              [59625]
12899780           [1142000, 582732, 973453, 736515, 1142000]
12899781    [141736, 199008, 57315, 194067, 199008, 199008...
12899782    [1669402, 1494780, 1494780, 1494780, 1494780, ...
12899783    [255297, 1114789, 255297, 300127, 198385, 3001...
                                  ...                        
12899848                                    [301163, 1379999]
12899849                                            [1551646]
12899850    [413690, 678776, 678776, 1729956, 1256317, 856...
12899851                                       [2209, 705659]
12899852    [1533519, 855511, 846336, 125597, 125597, 6633...
Name: aid, Length: 74, dtype: object

In [46]:
test_session_types

session
12899779                                                  [0]
12899780                                      [0, 0, 0, 0, 0]
12899781                    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
12899782    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...
12899783                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
                                  ...                        
12899848                                               [0, 0]
12899849                                                  [0]
12899850    [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...
12899851                                               [0, 0]
12899852    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: type, Length: 74, dtype: object

In [6]:
for i in range(0, sessions.shape[0], chunk_size):
    current_chunk = subsets.loc[sessions[i]:sessions[min(sessions.shape[0]-1, i+chunk_size-1)]].reset_index(drop=True)
    current_chunk = current_chunk.groupby('session', as_index=False).nth(list(range(-30,0))).reset_index(drop=True)
    consecutive_AIDs = current_chunk.merge(current_chunk, on='session')
    consecutive_AIDs = consecutive_AIDs[consecutive_AIDs.aid_x != consecutive_AIDs.aid_y]
    consecutive_AIDs['days_elapsed'] = (consecutive_AIDs.ts_y - consecutive_AIDs.ts_x) / (24 * 60 * 60)
    consecutive_AIDs = consecutive_AIDs[(consecutive_AIDs.days_elapsed >= 0) & (consecutive_AIDs.days_elapsed <= 1)]
    
    for aid_x, aid_y in zip(consecutive_AIDs['aid_x'], consecutive_AIDs['aid_y']):
        next_AIDs[aid_x][aid_y] += 1
    
del train, subset_of_train, subsets



In [28]:
session_types = ['clicks', 'carts', 'orders']
test_session_AIDs = subset_of_test.reset_index(drop=True).groupby('session')['aid'].apply(list)[1:2]
test_session_types = subset_of_test.reset_index(drop=True).groupby('session')['type'].apply(list)[1:2]

In [8]:
test_session_AIDs

session
12899779                                              [59625]
12899780           [1142000, 582732, 973453, 736515, 1142000]
12899781    [141736, 199008, 57315, 194067, 199008, 199008...
12899782    [1669402, 1494780, 1494780, 1494780, 1494780, ...
12899783    [255297, 1114789, 255297, 300127, 198385, 3001...
Name: aid, dtype: object

In [35]:


labels = []

no_data = 0
no_data_all_aids = 0
type_weight_multipliers = {0: 1, 1: 6, 2: 3}
for AIDs, types in zip(test_session_AIDs, test_session_types):
    
    if len(AIDs) >= 20:
        print("AID:", AIDs)
        print("type:", types)
        weights=np.logspace(0.1,1,len(AIDs),base=2, endpoint=True)-1 
        #erstellt Zahlenwerte zwischen 0.07 und 1 -> Gewichtungen, die letzte Aktion in der Session wird am höchsten gewichtet
        print("weights:", weights)
        aids_temp=defaultdict(lambda: 0)
        # erstellt ein dictionary mit Nullen
        print("aids_temp:", aids_temp)
        for aid,w,t in zip(AIDs,weights,types): 
            aids_temp[aid]+= w * type_weight_multipliers[t]
            print("aids_temp[aid]:", aids_temp[aid])
        
        # für jede aid in der Session wird ein Eintrag im dictionary erstellt. Dann wird mit den Gewichtungen für Zeit und Typ ein Wert eingefügt. 
        # Falls die aid in der Session mehrfach vorkommt, wird der Wert im dictionary entsprechend erhöht. 
            
        sorted_aids=[k for k, v in sorted(aids_temp.items(), key=lambda item: -item[1])] 
        #keys sind aids, values sind berechnete Ergebnisse. Wird sortiert nach -item[1]. Entspricht: Zahlenwerte absteigend
        print("sorted_aids:", sorted_aids)
        labels.append(sorted_aids[:20])
        # fügt die 20 größten Werte aus den sorted_aids in die Labels Liste ein. Entspricht den predictions für die Session.
       
    else:
        AIDs = list(dict.fromkeys(AIDs[::-1])) #erstellt eine Liste mit den AIDs der Session
        print("AIDs:", AIDs)
        AIDs_len_start = len(AIDs) 
        print("AIDs_len_start", AIDs_len_start)
      
        
        candidates = []
        for AID in AIDs:
            if AID in next_AIDs: candidates += [aid for aid, count in next_AIDs[AID].most_common(20)]
            # schlägt jede ID der Session im dictionary (next_AIDs, co-visitation matrix) nach.
            # Falls es für diese ID Einträge gibt, werden bis zu zwanzig häufigsten IDs zu den candidates hinzugefügt. 
            # die candidates können demnach Anzahl der AIDs * 20 lang sein.

            print("AID", AID)
            print("candidates", candidates)
           
            
        AIDs += [AID for AID, cnt in Counter(candidates).most_common(40) if AID not in AIDs]
        # die Liste der AIDs aus der Session wird jetzt mit IDs aus den candidates (aus dem dictionary) erweitert: 
        # Es werden die candidates genommen, die am häufigsten vorkommen und noch nicht in der Session vorgekommen sind.
        
        print("AIDs", AIDs)
        print("candidates", candidates)
        
        labels.append(AIDs[:20])
        # Die erweiterte Liste der Session AIDs wird für die prediction benutzt.
        # Das Ergebnis kann kleiner als zwanzig sein.
        # Ggf. weitere Matrix benutzen?

        if candidates == []: no_data += 1
        if AIDs_len_start == len(AIDs): no_data_all_aids += 1



AIDs: [1142000, 736515, 973453, 582732]
AIDs_len_start 4
AID 1142000
candidates [582732, 973453, 736515]
AID 736515
candidates [582732, 973453, 736515, 298280, 1515511, 1142000, 1360606, 797055]
AID 973453
candidates [582732, 973453, 736515, 298280, 1515511, 1142000, 1360606, 797055, 736515, 1142000]
AID 582732
candidates [582732, 973453, 736515, 298280, 1515511, 1142000, 1360606, 797055, 736515, 1142000, 973453, 736515, 1142000]
AIDs [1142000, 736515, 973453, 582732, 298280, 1515511, 1360606, 797055]
candidates [582732, 973453, 736515, 298280, 1515511, 1142000, 1360606, 797055, 736515, 1142000, 973453, 736515, 1142000]


In [31]:
next_AIDs

defaultdict(collections.Counter,
            {543308: Counter({961113: 1,
                      883849: 1,
                      701766: 1,
                      924751: 2,
                      168206: 1,
                      219033: 1,
                      171982: 1,
                      1319939: 1,
                      1349536: 3,
                      334392: 2,
                      165096: 1,
                      315914: 2,
                      1680276: 1,
                      1818905: 1,
                      1048797: 1,
                      341626: 1,
                      219925: 1,
                      843110: 1,
                      938007: 1,
                      1228848: 1,
                      1740927: 1,
                      161938: 1}),
             961113: Counter({883849: 1,
                      701766: 1,
                      924751: 2,
                      168206: 1,
                      219033: 1,
                      171982: 1,
                  

In [20]:
sorted_aids

[1674681,
 602722,
 1596098,
 45034,
 603159,
 1037537,
 1299062,
 1352725,
 794259,
 363336,
 654809,
 723956,
 406001,
 1099390,
 638410,
 1072927,
 530899,
 1494780,
 413962,
 229748,
 562753,
 1669402,
 1696036,
 479970,
 1033148,
 476063,
 1007613,
 1711180,
 829180,
 127404,
 975116,
 1344773,
 779477,
 987399,
 889671,
 740494,
 595994,
 834354]

In [18]:
sorted_aids

[834354,
 595994,
 740494,
 889671,
 987399,
 779477,
 1344773,
 975116,
 127404,
 829180,
 1711180,
 1007613,
 476063,
 1033148,
 479970,
 1696036,
 1669402,
 562753,
 229748,
 413962,
 1494780,
 530899,
 1072927,
 638410,
 1099390,
 406001,
 723956,
 654809,
 363336,
 794259,
 1352725,
 1299062,
 1037537,
 603159,
 45034,
 1596098,
 602722,
 1674681]

In [21]:
labels

[[1674681,
  602722,
  1596098,
  45034,
  603159,
  1037537,
  1299062,
  1352725,
  794259,
  363336,
  654809,
  723956,
  406001,
  1099390,
  638410,
  1072927,
  530899,
  1494780,
  413962,
  229748]]

In [17]:
aids_temp.values()

dict_values([2.70381569764724, 0.9415342346298132, 0.12133503273713808, 0.1315190467584635, 0.1417955524424579, 0.15216538980246397, 0.16262940648086266, 1.4708440241070397, 4.7053138154586955, 0.2163923899480118, 2.961835608717208, 2.182815559438255, 3.2466466975049197, 6.191834812668787, 0.43136469857942994, 0.4443644156438129, 3.2817968809446407, 0.4840762496333857, 0.4975546953197658, 3.194348242916396, 3.149279603696791, 3.2323737629344955, 0.5527037644960433, 0.5668054882868307, 0.6098837577117002, 5.144385428629587, 12.671315804076116, 0.6996260857281598, 5.098892770923411, 0.746356168683505, 0.7622166521151952, 0.7782211811540747, 1.605038684877593, 5.889348764642284, 2.734807695716773, 2.786891471107457, 2.8924823993495936, 3.0])

In [36]:
# >>> outputting results to CSV

labels_as_strings = [' '.join([str(l) for l in lls]) for lls in labels]

predictions = pd.DataFrame(data={'session_type': test_session_AIDs.index, 'labels': labels_as_strings})

prediction_dfs = []

for st in session_types:
    modified_predictions = predictions.copy()
    modified_predictions.session_type = modified_predictions.session_type.astype('str') + f'_{st}'
    prediction_dfs.append(modified_predictions)

submission = pd.concat(prediction_dfs).reset_index(drop=True)
submission.to_csv('submission.csv', index=False)

print(f'Test sessions that we did not manage to extend based on the co-visitation matrix: {no_data_all_aids}')

Test sessions that we did not manage to extend based on the co-visitation matrix: 0


In [40]:
test_session_AIDs

session
12899780    [1142000, 582732, 973453, 736515, 1142000]
Name: aid, dtype: object

In [39]:
submission

,session_type,labels
0,12899780_clicks,1142000 736515 973453 582732 298280 1515511 13...
1,12899780_carts,1142000 736515 973453 582732 298280 1515511 13...
2,12899780_orders,1142000 736515 973453 582732 298280 1515511 13...


In [37]:
labels_as_strings

['1142000 736515 973453 582732 298280 1515511 1360606 797055']

In [38]:
predictions.head()

,session_type,labels
0,12899780,1142000 736515 973453 582732 298280 1515511 13...


In [49]:
candidates

[705659]

In [48]:
AIDs

[1533519,
 855511,
 846336,
 125597,
 125597,
 663333,
 663333,
 663333,
 663333,
 663333,
 663333,
 663333,
 663333,
 663333,
 663333,
 663333,
 663333,
 1580901,
 1533519,
 701401,
 701401,
 1579739,
 372283,
 199292]

In [ ]:
consecutive_AIDs